# 라이브러리

In [ ]:
import numpy as np
import cv2
import os
from glob import glob
import matplotlib.pyplot as plt

# 이미지 합성

In [ ]:
# train 원본 이미지
img_path = '/kaggle/input/dacon-blocks/train'
images = glob(os.path.join(img_path, '*'))
images.sort()

## train_316
* train의 배경 중 흰 격자 배경이 아닌 이미지가 섞여 있었다. 그 이미지의 배경을 활용하여 새로운 train data를 만들었다.

In [ ]:
#bg1 = TRAIN_31668 배경추출
bg1_path = './TRAIN_31668.jpg'
bg1_img = cv2.imread(bg1_path)

#bg2 = TRAIN_31690 배경추출
bg2_path = './TRAIN_31690.jpg'
bg2_img = cv2.imread(bg2_path)

#격자 이미지 
bgwhite_path = "./whitegrid_bg.jpg"

img_bin = np.array(bg1_img[:,:,0] < 130, dtype=np.uint8)*255 
img_blur = cv2.medianBlur(img_bin, 5) # 잡음 제거 (커널 5*5)
img_filter = bg1_img.copy() 
img_filter[img_blur == 0] = 0 # 배경을 0으로 만들기
bg1 = bg1_img - img_filter # 블록 부분 빼기 (배경만 만들기)
img_back = cv2.resize(cv2.imread(bgwhite_path), bg1.shape[::-1][1:]) # 입힐 배경(wg) 사이즈에 맞게 가져오기
bg1 += img_back # 제거된 블록 부분에 whitegrid 입히기 
# plt.imshow(bg1) 

img_bin = np.array(bg2_img[:,:,0] < 130, dtype=np.uint8)*255
img_blur = cv2.medianBlur(img_bin, 5)
img_filter = bg2_img.copy()
img_filter[img_blur == 0] = 0
bg2 = bg2_img - img_filter
img_back = cv2.resize(cv2.imread(bgwhite_path), bg2.shape[::-1][1:])
bg2 += img_back

# plt.imshow(bg2)

In [ ]:
#Train에 홀짝으로 Train 316 배경 입히기
bgs = [bg1, bg2]

save_path = "./train_316/"

cnt=0
for i in range(len(images)):
    cnt += 1
    img = cv2.imread(images[i])
#   plt.imshow(img)

    img_bin = np.array(img[:,:,0] < 155, dtype=np.uint8)*255
    img_blur = cv2.medianBlur(img_bin, 5)
    img_filter = img.copy()
    img_filter[img_blur == 0] = 0 

    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(img_blur) #객체 추출
    stats = np.array(stats) 
    stats[0, -1] = 0
    label_idx = np.argmax(stats[:,-1])
    img_mask = labels == label_idx 
    img_final = img_filter.copy()
    img_final[~img_mask] = (0,0,0) # 블록만 추출
    plt.imshow(img_final)
    
    h = i%len(bgs)
    img_back = cv2.resize(bgs[h], img.shape[::-1][1:])
    img_back[img_mask] = (0,0,0) # 블록이 아닌거만 추출
    img_filter += img_back
#   plt.imshow(img_filter)
    cv2.imwrite(save_path+images[i].split('/')[-1],img_filter)
    print(cnt)

## train_new
* 새로운 이미지를 가져와서 합성

In [ ]:
#bg_new 가져오기
bg_path = './bg_new'
bgs = glob(os.path.join(bg_path, '*'))

In [ ]:
#train_new
save_path = "./train_new/"

cnt=0
for i in range(len(images)):
    cnt += 1
    img = cv2.imread(images[i])
#    plt.imshow(img)

    img_bin = np.array(img[:,:,0] < 155, dtype=np.uint8)*255
    img_blur = cv2.medianBlur(img_bin, 5)
    img_filter = img.copy()
    img_filter[img_blur == 0] = 0

    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(img_blur)
    stats = np.array(stats)
    stats[0, -1] = 0
    label_idx = np.argmax(stats[:,-1])
    img_mask = labels == label_idx
    img_final = img_filter.copy()
    img_final[~img_mask] = (0,0,0)
    
    h = i%len(bgs)
    img_back = cv2.resize(cv2.imread(bgs[h]), img.shape[::-1][1:])
    img_back[img_mask] = (0,0,0)
    img_filter += img_back
#    plt.imshow(img_filter)
    cv2.imwrite(save_path+images[i].split('/')[-1],img_filter)
    print(cnt)